In [9]:
import pandas as pd
from openai import OpenAI
import os
from dotenv import load_dotenv
import csv

import markdown2
import re

## GPT prompt engineering

In [ ]:
# Load environment variables from .env file
load_dotenv()

# Access the API key using the variable name defined in the .env file
client = OpenAI()
client.api_key = os.getenv("OPENAI_API_KEY")
print(client.api_key)

In [3]:
def read_tourism_spots(csv_file_path):
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_string = file.read()
    return csv_string

# CSV 파일 경로 설정
csv_file_path = './data/small_set.csv'

kr_locations = read_tourism_spots(csv_file_path)

In [4]:
print(kr_locations)

POI_ID,POI_NM,CTPRVN_NM,SIGNGU_NM,LEGALDONG_NM,LC_LO,LC_LA
7231419,임랑해수욕장,부산광역시,기장군,장안읍,129.2642611,35.31869433
7232777,오이도해양단지,경기도,시흥시,정왕동,126.6902931,37.34640271
11324755,광안리카페거리,부산광역시,수영구,민락동,129.1213895,35.15521088
16758706,구로시장떡볶이골목,서울특별시,구로구,구로동,126.8855426,37.48881811
20868088,덕천동젊음의거리,부산광역시,북구,덕천동,129.0070903,35.20989557
21085658,동대문패션타운,서울특별시,중구,을지로6가,127.0086616,37.56866403
21085686,을지로타일상가,서울특별시,중구,을지로2가,126.98898,37.56637497
21085883,사강회센터,경기도,화성시,송산면,126.7362366,37.21304632
21086097,까치울음식테마마을,경기도,부천시,작동,126.8138061,37.51514031



## The first analysis

In [5]:
# prompt engineering depends on the system's content
system_cont_ls = ["You are the best travel planner who can make efficient travel plans.\nPlease write based on the data I presented.", 
"You are the best travel planner who can make efficient travel plans.\nPlease write based on the data I presented.\nPlease provide information on the travel route, tourist destination (POI_NM), and province (CTPRVN_NM).",
"You are the best travel planner who can make efficient travel plans.\nPlease write based on the data I presented.\nPlease provide information on the travel route, tourist destination (POI_NM), and province (CTPRVN_NM).\nPlease organize your travel route efficiently considering latitude (LC_LA) longitude (LC_LO).",
"You are the best travel planner who can make efficient travel plans.\nPlease write based on the data I presented.\nPlease provide information on the travel route, tourist destination (POI_NM), and province (CTPRVN_NM).\nPlease organize your travel route efficiently considering latitude (LC_LA) longitude (LC_LO).\nPlease provide the name of your area in English (Korean). The basic number of travel days is 2 days.",
"You are the best travel planner who can make efficient travel plans.\nPlease write based on the data I presented.\nPlease provide information on the travel route, tourist destination (POI_NM), and province (CTPRVN_NM).\nPlease organize your travel route efficiently considering latitude (LC_LA) longitude (LC_LO).\nPlease provide the name of your area in English (Korean). The basic number of travel days is 2 days.\nIf the maximum length is insufficient, please include only POI_NM, CTPRVN_NM by day.",
"You are the best travel planner who can make efficient travel plans.\nPlease write based on the data I presented.\nPlease provide information on the travel route, tourist destination (POI_NM), and province (CTPRVN_NM).\nPlease organize your travel route efficiently considering latitude (LC_LA) longitude (LC_LO).\nPlease provide the name of your area in English (Korean). The basic number of travel days is 2 days.\nIf the maximum length is insufficient, please include only POI_NM, CTPRVN_NM by day.\nDon't include the explain."]

user_cont_ls = ["Make a plan for a trip to the sea in Korea."
                , "I want to find out about Korean fashion."
                , "Make travel plans for restaurants"
                , "I want to go to historical places"
                , "I want to travel on sidewalks as much as possible."
                , "I want to go to different regions."
                , "I'm interested in places related to food. I'm going to stay in Seoul. Please recommend places and attractions around me."
                , "I am interested in Korean fashion street and history."
                , "I'm planning a trip with my elderly parents. I'm planning to travel for 3 days. Please write a travel plan that I can travel comfortably."]

In [6]:
# GPT-3.5 Turbo API를 사용하여 여행 계획을 생성하는 함수
def generate_travel_plan(tourism_spots, sys_cont, user_cont):

    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        temperature=1,
        max_tokens=512,
        top_p=0.7,
        frequency_penalty=0,
        presence_penalty=0,
        messages=[
            { "role": "system", "content": """See Korea tourist attraction:\n{}. {}""".format(tourism_spots, sys_cont)},
            { "role": "user",  "content": f"{user_cont}"},
        ],
        stream=False,
    )
    # 응답에서 생성된 계획 추출
    generated_plan = completion.choices[0].message

    return generated_plan

In [7]:
assistant_answer_ls = []
for i in range(len(system_cont_ls)):
    travel_plan = generate_travel_plan(kr_locations, system_cont_ls[i], user_cont_ls[0]).content
    assistant_answer_ls.append(travel_plan) # save the anwers

In [ ]:
j=0
for i in assistant_answer_ls:
    print(f'system {j+1}:  \n{system_cont_ls[j]}\n')
    print(f'answer {j+1}:  \n{i}')
    j+=1
    print("\n------------------------------------------------------------------------------------------------\n")

In [9]:
with open("./html/travel_plans_v1.html", "w", encoding="utf-8") as file:
# with open("./html/travel_plans_v2.html", "w", encoding="utf-8") as file:
    file.write("<html>\n<head>\n<title>The First: Travel Plans</title>\n</head>\n<body>\n")
    j = 0
    for i in assistant_answer_ls:
        i_with_line_break = re.sub(r'\n\s*-', '<br> -', i)
        i_with_line_break = re.sub(r'(\d+)\. ', r'\n\1. ', i_with_line_break)
        
        html_content = markdown2.markdown(i_with_line_break)
        
        file.write(f"<h3>System {j+1}</h3>\n{(system_cont_ls[j])}\n")
        file.write(f"<h2>Answer {j+1}</h2>\n{html_content}\n<hr>\n")
        j += 1
    file.write("</body>\n</html>")


## The second analysis

In [4]:
test_set_locations = read_tourism_spots("./data/test_set.csv")

In [5]:
improve_system_v1 = "You are the best travel planner who can make efficient travel plans.\nPlease write based on the data I presented.\nPlease provide information on the travel route, tourist destination (POI_NM), and province (CTPRVN_NM).\nthe name of tourist destination(POI_NM) and write it by Day.\nWrite it by Day.\nPlease organize your travel route efficiently considering latitude (LC_LA) longitude (LC_LO)."
improve_system_v2 = "You are the best travel planner who can make efficient travel plans. Please write based on the data I presented. Please provide information on the travel route, tourist destination (POI_NM), and province (CTPRVN_NM). Write it by Day including 3 tourist destination in english and korea. Please organize your travel route efficiently considering latitude (LC_LA) longitude (LC_LO)."
improve_system_v3 = "You are the best travel planner who can make efficient travel plans. Please write based on the data I presented. Please provide information on the travel route, tourist destination (POI_NM), and province (CTPRVN_NM). Please write in a readable manner, including three tourist attractions per day in English and Korean. Please organize your travel route efficiently considering latitude (LC_LA) longitude (LC_LO)."
improve_system_v4 = "You are the best travel planner who can make efficient travel plans. Please write based on the data I presented. Please provide information on the travel route, tourist destination (POI_NM), and province (CTPRVN_NM). Please fill out three tourist attractions per day in English and Korean. Write the tourist attractions in bold font. Please organize your travel route efficiently considering latitude (LC_LA) longitude (LC_LO)."
improve_system_v5 = "You are the best travel planner who can make efficient travel plans. Please write based on the data I presented. Please provide information on the travel route, tourist destination (POI_NM), and province (CTPRVN_NM). Please fill out three tourist attractions in English and Korean per day. Write the tourist attractions in bold font. Please organize your travel route efficiently considering latitude (LC_LA) longitude (LC_LO)."

In [11]:
def generate_improve_travel_plan(tourism_spots, sys_cont, user_cont):

    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        temperature=1,
        max_tokens=512,
        top_p=0.7,
        frequency_penalty=0,
        presence_penalty=0,
        messages=[
            { "role": "system", "content": """See this desciption:\n{}. {}.""".format(tourism_spots, sys_cont)},
            { "role": "user",  "content": f"{user_cont}"},
        ],
        stream=False,
    )
    generated_plan = completion.choices[0].message

    return generated_plan

In [15]:
improve_answer_ls = []
user_cont = "Make a plan for a trip in Korea."

version_num = 4 # it can be changed 1-5
improve_system_file = improve_system_v4 # it can be changed improve_system_v1 - v5

improve_answer = generate_improve_travel_plan(kr_locations, improve_system_file, user_cont).content
improve_answer_ls.append(improve_answer) # save the anwers

In [20]:
with open(f"./html/travel_plans_improve_v{version_num}.html", "w", encoding="utf-8") as file:
    file.write("<html>\n<head>\n<title>Improve Travel Plans</title>\n</head>\n<body>\n")
    j = 0
    for i in improve_answer_ls:
        
        i_with_line_break = re.sub(r'\n\s*-', '<br> -', i)
        i_with_line_break = re.sub(r'(\d+)\. ', r'\n\1. ', i_with_line_break)
        
        html_content = markdown2.markdown(i_with_line_break)
        
        file.write(f"<h3>Improve System {version_num}</h3>\n{improve_system_file}\n")
        file.write(f"<h2>Answer {version_num}</h2>\n{html_content}\n<hr>\n")
        j += 1
    file.write("</body>\n</html>")
